In [1]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, f1_score
from sklearn_crfsuite import CRF
from scipy.sparse import hstack

In [ ]:
! pip install sklearn_crfsuite

# Entity recognition
Мы будем использовать часть корпуса GMB (только часть, чтобы быстро работало). В корпусе размечены следующие тэги:

Tag | Label meaning | Example Given
--- | ------------- | -------------
geo | Geographical Entity | London
org | Organization | ONU
per | Person | Bush
gpe | Geopolitical Entity | British
tim | Time indicator | Wednesday
art | Artifact | Chrysler
eve | Event | Christmas
nat | Natural Phenomenon | Hurricane
O | No-Label | the

Тэги размечены по системе BIO. В таблице, которую мы будем использовать, есть уже все признаки по каждому слову и его контексту.

Читаем и смотрим данные:

In [2]:
data = pd.read_csv('ner_short.csv')
print('\n'.join(data.columns))
data.head()

Unnamed: 0
lemma
next-lemma
next-next-lemma
next-next-pos
next-next-shape
next-next-word
next-pos
next-shape
next-word
pos
prev-iob
prev-lemma
prev-pos
prev-prev-iob
prev-prev-lemma
prev-prev-pos
prev-prev-shape
prev-prev-word
prev-shape
prev-word
sentence_idx
shape
word
tag


,Unnamed: 0,lemma,next-lemma,next-next-lemma,next-next-pos,next-next-shape,next-next-word,next-pos,next-shape,next-word,...,prev-prev-lemma,prev-prev-pos,prev-prev-shape,prev-prev-word,prev-shape,prev-word,sentence_idx,shape,word,tag
0,0,thousand,of,demonstr,NNS,lowercase,demonstrators,IN,lowercase,of,...,__start2__,__START2__,wildcard,__START2__,wildcard,__START1__,1.0,capitalized,Thousands,O
1,1,of,demonstr,have,VBP,lowercase,have,NNS,lowercase,demonstrators,...,__start1__,__START1__,wildcard,__START1__,capitalized,Thousands,1.0,lowercase,of,O
2,2,demonstr,have,march,VBN,lowercase,marched,VBP,lowercase,have,...,thousand,NNS,capitalized,Thousands,lowercase,of,1.0,lowercase,demonstrators,O
3,3,have,march,through,IN,lowercase,through,VBN,lowercase,marched,...,of,IN,lowercase,of,lowercase,demonstrators,1.0,lowercase,have,O
4,4,march,through,london,NNP,capitalized,London,IN,lowercase,through,...,demonstr,NNS,lowercase,demonstrators,lowercase,have,1.0,lowercase,marched,O


In [8]:
data.tag.value_counts(normalize=True )

O        0.846915
B-geo    0.035322
B-tim    0.018845
B-org    0.018625
I-per    0.017160
B-gpe    0.016537
B-per    0.016404
I-org    0.015071
I-geo    0.007219
I-tim    0.005703
B-art    0.000513
B-eve    0.000408
I-eve    0.000325
I-gpe    0.000311
I-art    0.000298
B-nat    0.000243
I-nat    0.000101
Name: tag, dtype: float64

## Логистическая регрессия

Векторизуем наши фичи и объединим матрицы в одну. Сейчас я буду использовать лемму, часть речи и попробую добавить тэг предыдущего слова.

In [11]:
w_vectorizer = TfidfVectorizer()
pos_vectorizer = CountVectorizer(token_pattern='.+')
tag_vect = CountVectorizer(token_pattern='.+')

w_vect = w_vectorizer.fit_transform(data.lemma)
pos_vect = pos_vectorizer.fit_transform(data.pos)
prev_tag_vect = tag_vect.fit_transform(data['prev-iob'])

#X = hstack((w_vect, pos_vect))
X = hstack((w_vect, pos_vect, prev_tag_vect))
y = data.tag

Разделим выборку на обучающую и тестовую.

In [12]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

Обучим логистическую регрессию с дефолтными параметрами.

In [13]:
logit = LogisticRegression(random_state=42)
logit.fit(X_train, y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=42, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

Посмотрим качество нашей модели. Метрика - f1-score с macro усреднением - из-за дисбаланса классов (macro усреднение берет все классы с равным весом).

In [15]:
y_pred = logit.predict(X_test)
print(f1_score(y_test, y_pred, average='macro'))
print(classification_report(y_test, y_pred))

C:\Users\PC1\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


0.7497133497123338


C:\Users\PC1\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


             precision    recall  f1-score   support

      B-art       0.67      0.03      0.06        61
      B-eve       1.00      0.19      0.31        59
      B-geo       0.72      0.92      0.81      5145
      B-gpe       0.97      0.82      0.89      2379
      B-nat       0.00      0.00      0.00        36
      B-org       0.82      0.51      0.63      2767
      B-per       0.85      0.77      0.81      2305
      B-tim       0.98      0.73      0.84      2639
      I-art       1.00      0.98      0.99        41
      I-eve       1.00      0.98      0.99        47
      I-geo       0.98      0.95      0.97      1090
      I-gpe       1.00      0.67      0.80        52
      I-nat       1.00      0.79      0.88        14
      I-org       0.95      0.91      0.93      2156
      I-per       0.94      0.99      0.97      2525
      I-tim       0.91      0.83      0.87       828
          O       0.99      0.99      0.99    121936

avg / total       0.97      0.97      0.97  

## CRF

Для CRF нужно по-другому предобработать данные. На вход подается список предложений, где каждое предложение - это список словарей, а в словаре уже лежат признаки для каждого слова. И тут не нужно кодировать строки, подаем их как они есть.

In [17]:
sents = data.groupby(['sentence_idx'])
crf_data = []
crf_y = []
for name, sent in sents:
    crf_y.append(sent['tag'].tolist())
    sent = sent[['lemma', 'pos', 'prev-iob']]
    crf_data.append(sent.to_dict(orient='records'))
X_train, X_test, y_train, y_test = train_test_split(crf_data, crf_y, test_size=0.33, random_state=42)

Фиттим модель.

In [20]:
crf = CRF()
crf.fit(X_train, y_train)

CRF(algorithm=None, all_possible_states=None, all_possible_transitions=None,
  averaging=None, c=None, c1=None, c2=None, calibration_candidates=None,
  calibration_eta=None, calibration_max_trials=None, calibration_rate=None,
  calibration_samples=None, delta=None, epsilon=None, error_sensitive=None,
  gamma=None, keep_tempfiles=None, linesearch=None, max_iterations=None,
  max_linesearch=None, min_freq=None, model_filename=None,
  num_memories=None, pa_type=None, period=None, trainer_cls=None,
  variance=None, verbose=False)

Смотрим скор.

In [22]:
crf = CRF()
crf.fit(X_train, y_train)
y_pred = crf.predict(X_test)
print(f1_score([x for y in y_test for x in y], [x for y in y_pred for x in y], average='macro'))

0.7924727450142887
0.7924727450142887
0.7924727450142887
0.7924727450142887
0.7924727450142887


# Задание

Немного подучиться машинному обучению, подумать и к вечеру 11го января (или раньше) сделать что-нибудь, у чего скор будет лучше. Чем лучше - тем лучше :) Идеи:
* другие модели (ансамбли и нейросети тоже можно)
* подключить еще фичи
* получить новые фичи из имеющихся
* семантические вектора - попробовать использовать предобученные, обучить самим на этих данных
* тюнинг гиперпараметров

Полные данные (ок. миллиона строк) тоже лежат в репозитории. Их нужно распаковать и немного по-другому читать:

In [3]:
data = pd.read_csv('ner.csv', encoding='ISO-8859-1', error_bad_lines=False)
len(data)

b'Skipping line 281837: expected 25 fields, saw 34\n'


1050795

## Дополнительное задание
В 2015 году на Диалоге проводили соревнование по сентименту. [Вот тут лежат статьи](http://www.dialog-21.ru/evaluation/2015/sentiment/) (и ссылка на материалы тоже лежит). Попробуйте:
* скачать [этот файл](https://drive.google.com/file/d/0B7y8Oyhu03y_YV9ZU0cwZGVtenM/view)
* распарсить его (`lxml.etree`, с этим могу помочь)
* предсказать по тексту сентимент (атрибут `sentiment`) по аспектам, которые указаны в `<categories>` - смотрите по метрике f-score с макро-усреднением - на соревновании это было задание Д (а основное - выделить аспекты)
* подумать, как это можно улучшить
* возможно, стоит брать не текст целиком а уметь находить конкретные аспекты и их сентимент в тексте -> задача выделения аспектов (сложная)
* вдохновляться статьями можно, смотреть скоры в обзорной статье и в табличке на гуглдиске можно